# Question Answering with Sources

This notebook walks through how to use LangChain for question answering with sources over a list of documents. It covers three different chain types: `stuff`, `map_reduce`, and `refine`. For a more in depth explanation of what these chain types are, see [here](../combine_docs.md).

## Prepare Data
First we prepare the data. For this example we do similarity search over a vector database, but these documents could be fetched in any manner (the point of this notebook to highlight what to do AFTER you fetch the documents).

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document

In [2]:
with open('../../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [3]:
docsearch = FAISS.from_texts(texts, embeddings, metadatas=[{"source": i} for i in range(len(texts))])

query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)

In [4]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

## The `stuff` Chain

This sections shows results of using the `stuff` Chain to do question answering with sources.

In [5]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

In [6]:
query = "What did the president say about Justice Breyer"
chain.combine_and_parse(**{"docs": docs, "question": query})

{'answer': ' The president thanked Justice Breyer for his service.',
 'sources': '30-pl'}

## The `map_reduce` Chain

This sections shows results of using the `map_reduce` Chain to do question answering with sources.

In [7]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_reduce")

In [8]:
query = "What did the president say about Justice Breyer"
chain.combine_and_parse(**{"docs": docs, "question": query})

{'answer': ' The president thanked Justice Breyer for his service.',
 'sources': '30-pl'}

**Intermediate Steps**

We can also return the intermediate steps for `map_reduce` chains, should we want to inspect them. This is done with the `return_map_steps` variable.

In [9]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_reduce", return_map_steps=True)

In [10]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'map_steps': [' "Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service."',
  ' None',
  ' None',
  ' None'],
 'output_text': ' The president thanked Justice Breyer for his service.\nSOURCES: 30-pl'}

## The `refine` Chain

This sections shows results of using the `refine` Chain to do question answering with sources.

In [11]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine")

In [12]:
query = "What did the president say about Justice Breyer"
chain.combine_and_parse(**{"docs": docs, "question": query})

{'answer': "\n\nThe president said that he was honoring Justice Breyer for his dedication to serving the country and that he was a retiring Justice of the United States Supreme Court. He also thanked him for his service and praised his career as a top litigator in private practice, a former federal public defender, and a family of public school educators and police officers. He noted Justice Breyer's reputation as a consensus builder and the broad range of support he has received from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. He also highlighted the importance of securing the border and fixing the immigration system in order to advance liberty and justice, and mentioned the new technology, joint patrols, dedicated immigration judges, and commitments to support partners in South and Central America that have been put in place. He also expressed his commitment to the LGBTQ+ community, noting the need for the bipartisan Equality Act and the imp

**Intermediate Steps**

We can also return the intermediate steps for `refine` chains, should we want to inspect them. This is done with the `return_refine_steps` variable.

In [13]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine", return_refine_steps=True)

In [14]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'refine_steps': ['\nThe president said that he was honoring Justice Breyer for his dedication to serving the country and that he was a veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court.',
  '\n\nThe president said that he was honoring Justice Breyer for his dedication to serving the country, his experience as a veteran, Constitutional scholar, former top litigator in private practice, former federal public defender, and retiring Justice of the United States Supreme Court. He also noted that Justice Breyer came from a family of public school educators and police officers, and that he was a consensus builder who had received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. The president also highlighted his commitment to advancing liberty and justice by securing the border and fixing the immigration system, including installing new technology, setting up joint patrols with Mexico a